# HR Attendance Processing Notebook

- The earlier notebooks worked for faculties while it created some problem for staffs. Trying to solve the same

In [740]:
import warnings
import io
import pandas as pd
import numpy as np
#import gspread
#from google.oauth2.service_account import Credentials
#from utility import connect_gsheet, get_dataframe, gs_client
from utility_attendance import (
    stepwise_file_upload, split_file, merge_files, move_columns,
    weighted_sum_and_replace_columns, calculate_leave_summary_with_wd_leaves,
    calculate_working_days
)
from datetime import datetime, timedelta, date
warnings.filterwarnings("ignore")

In [741]:
#from datetime import datetime, date

In [742]:
%run utility_attendance.py

In [743]:
no_working_days = 20

In [744]:
HOLIDAY_LIST = ['29-sep-2025','30-sep-2025','01-oct-2025','02-oct-2025','03-oct-2025','06-oct-2025','18-oct-2025',\
                '20-oct-2025','21-oct-2025','04-nov-2025','25-dec-2025']
misc_working_days = ''

In [745]:
misc_holidays = '17-sep-2025'
misc_holidays_splitted = [d.strip() for d in misc_holidays.split(',')]  # remove spaces

# Extend HOLIDAY_LIST with new dates
HOLIDAY_LIST.extend(misc_holidays_splitted)

print(HOLIDAY_LIST)

['29-sep-2025', '30-sep-2025', '01-oct-2025', '02-oct-2025', '03-oct-2025', '06-oct-2025', '18-oct-2025', '20-oct-2025', '21-oct-2025', '04-nov-2025', '25-dec-2025', '17-sep-2025']


## Step 1: Read files and merge

In [746]:
# 
df_gimt = pd.read_excel('./data/GIMT_MonthlyAttendanceSummaryReport (28).xlsx')
df_admin = pd.read_excel('./data/Admin_MonthlyAttendanceSummaryReport(45).xlsx')
df_gips = pd.read_excel('./data/GIPS_MonthlyAttendanceSummaryReport(45).xlsx')
df_leave_erp = pd.read_csv('./data/Leave report sep25.csv',skiprows=6, encoding='windows-1252')
exempted_file = pd.ExcelFile('./data/template_exempted_sep.xlsx')

### Reading EMP data from ERP
df_emp = pd.read_csv('./data/emp_master_data.csv', skiprows=6, encoding='windows-1252')

In [747]:
df_gimt_all, df_gimt_in, df_gimt_out = split_file(df_gimt)
df_gips_all, df_gips_in, df_gips_out = split_file(df_gips)
df_admin_all, df_admin_in, df_admin_out = split_file(df_admin)
#df_gimt_merged = merge_files(df_gimt_in, df_gimt_out, no_working_days)
#df_gips_merged = merge_files(df_gips_in, df_gips_out, no_working_days)

In [748]:
df_gimt_in = pad_month_in_columns(df_gimt_in, 'clock_in')
df_gimt_out = pad_month_in_columns(df_gimt_out, 'clock_out')

df_gips_in = pad_month_in_columns(df_gips_in, 'clock_in')
df_gips_out = pad_month_in_columns(df_gips_out, 'clock_out')

df_admin_in = pad_month_in_columns(df_admin_in, 'clock_in')
df_admin_out = pad_month_in_columns(df_admin_out, 'clock_out')

In [749]:
result = df_gimt_in[df_gimt_in['Emp Id']=='GCU010013']
result

,Emp Id,Names,Present,clock_in_08_26,clock_in_08_27,clock_in_08_28,clock_in_08_29,clock_in_08_30,clock_in_08_31,clock_in_09_01,...,clock_in_09_11,clock_in_09_12,clock_in_09_13,clock_in_09_14,clock_in_09_15,clock_in_09_16,clock_in_09_17,clock_in_09_18,clock_in_09_19,clock_in_09_20
0,GCU010013,Ms. Raisa,17.0,08:39:00,08:39:00,12:17:00,08:40:00,08:39:00,0,08:41:00,...,08:39:00,0,08:37:00,0,08:39:00,08:42:00,0,0,08:38:00,0


#### Holidays in aug-sep
- 08_31 (sun), 09_06 (sat), 09_07 (sun), 09_14 (sun), 09_20 (sat)

In [750]:
HOLIDAY_LIST

['29-sep-2025',
 '30-sep-2025',
 '01-oct-2025',
 '02-oct-2025',
 '03-oct-2025',
 '06-oct-2025',
 '18-oct-2025',
 '20-oct-2025',
 '21-oct-2025',
 '04-nov-2025',
 '25-dec-2025',
 '17-sep-2025']

### 1.1 Removing holidays from the three dataframes

In [751]:
df_gimt_in.columns

Index(['Emp Id', 'Names', 'Present', 'clock_in_08_26', 'clock_in_08_27',
       'clock_in_08_28', 'clock_in_08_29', 'clock_in_08_30', 'clock_in_08_31',
       'clock_in_09_01', 'clock_in_09_02', 'clock_in_09_03', 'clock_in_09_04',
       'clock_in_09_05', 'clock_in_09_06', 'clock_in_09_07', 'clock_in_09_08',
       'clock_in_09_09', 'clock_in_09_10', 'clock_in_09_11', 'clock_in_09_12',
       'clock_in_09_13', 'clock_in_09_14', 'clock_in_09_15', 'clock_in_09_16',
       'clock_in_09_17', 'clock_in_09_18', 'clock_in_09_19', 'clock_in_09_20'],
      dtype='object')

In [752]:
holidays = detect_holidays_staffs(df_gimt_in, year=None, misc_holidays=HOLIDAY_LIST, misc_working_days="", verbose=True)

📅 Attendance period: 26-Aug-2025 → 20-Sep-2025
📆 Calendar holidays: ['clock_in_08_31', 'clock_in_09_06', 'clock_in_09_07', 'clock_in_09_14', 'clock_in_09_20']
➕ Manual holidays (within range): ['clock_in_09_17']
➖ Manual working days: []
🎯 Final holidays: ['clock_in_08_31', 'clock_in_09_06', 'clock_in_09_07', 'clock_in_09_14', 'clock_in_09_17', 'clock_in_09_20']


In [753]:
#08_31 (sun), 09_06 (sat), 09_07 (sun), 09_14 (sun), 09_20 (sat)

In [754]:
# for clock_in we got from the 'holidays' we need to work out for clock_out only
cols_to_delete_in = holidays
cols_to_delete_out = [ c.replace('clock_in','clock_out')  for c in holidays]

In [755]:
#cols_to_delete_out

In [756]:
df_gimt_in = df_gimt_in.drop(columns = cols_to_delete_in, axis=1)
df_gimt_out = df_gimt_out.drop(columns = cols_to_delete_out, axis=1)

df_gips_in = df_gips_in.drop(columns = cols_to_delete_in, axis=1)
df_gips_out = df_gips_out.drop(columns = cols_to_delete_out, axis=1)

df_admin_in = df_admin_in.drop(columns = cols_to_delete_in, axis=1)
df_admin_out = df_admin_out.drop(columns = cols_to_delete_out, axis=1)

In [757]:
no_days_in_hand = len(df_gimt_in.columns) - 3
no_days_in_hand 

20

In [758]:
emp_df = df_emp[['Employee ID','Name','Designation','Department']]
emp_df = emp_df.rename(columns={'Employee ID':'Emp Id'})

In [759]:
emp_df.head()

,Emp Id,Name,Designation,Department
0,APPLICANT_PORTAL_USER,APP_FIRST_NAME APP_MIDDLE_NAME APP_LAST_NAME,-,-
1,mis,MIS Admin,-,Administration
2,EMP4,Lokesh Yadav,-,School of Allied Health Sciences
3,EMP5,Birshu Sarma,-,School Of Engineering and Technology
4,GCU020041,Bhabajit Baruah,Assistant Professor,School Of Engineering and Technology


In [760]:
#misc_holidays='17-sep-2025' 
#misc_working_days=''


In [761]:
#df_admin_merged = merge_files(df_admin_in, df_admin_out, no_working_days)  merge_files_staffs
df_gimt_merged = merge_files_staffs(df_gimt_in, df_gimt_out, emp_df, no_working_days, HOLIDAY_LIST, misc_working_days) 
df_gips_merged = merge_files_staffs(df_gips_in, df_gips_out, emp_df, no_working_days, HOLIDAY_LIST, misc_working_days) 
df_admin_merged = merge_files_staffs(df_admin_in, df_admin_out, emp_df, no_working_days, HOLIDAY_LIST, misc_working_days) 

In [762]:
df_gimt_merged.head()

,Emp Id,Name,Designation,Department,late_flags,early_flags,half_day_flags,AM_abs,PM_abs,days_abs,No_of_AM_abs,No_of_PM_abs,actual_No_of_late,actual_half_day,actual_full_day,Working Days,Present,Absent
0,GCU010013,Ms. Raisa,-,Administration,08_28,"08_27, 09_13",,,,"09_12, 09_18",0,0,1,0,2,20,18.0,2.0
1,GCU010017,Ms. Ananya,-,Administration,08_27,,,,,09_03,0,0,1,0,1,20,19.0,1.0
2,GCU020001,Dr. Lipi,Assistant Professor & HoD (I/c),School of Natural Science,,,,,,"08_26, 09_11",0,0,0,0,2,20,18.0,2.0
3,GCU020002,Dr. Swarnali,Assistant Professor & HoD (I/c),School of Natural Science,,,,,,"08_28, 08_29",0,0,0,0,2,20,18.0,2.0
4,GCU020003,Dr. Kajal,Assistant Professor,School of Natural Science,,,,,,,0,0,0,0,0,20,20.0,0.0


In [763]:
df_admin_merged.columns

Index(['Emp Id', 'Name', 'Designation', 'Department', 'late_flags',
       'early_flags', 'half_day_flags', 'AM_abs', 'PM_abs', 'days_abs',
       'No_of_AM_abs', 'No_of_PM_abs', 'actual_No_of_late', 'actual_half_day',
       'actual_full_day', 'Working Days', 'Present', 'Absent'],
      dtype='object')

In [764]:
df_gips_merged.columns

Index(['Emp Id', 'Name', 'Designation', 'Department', 'late_flags',
       'early_flags', 'half_day_flags', 'AM_abs', 'PM_abs', 'days_abs',
       'No_of_AM_abs', 'No_of_PM_abs', 'actual_No_of_late', 'actual_half_day',
       'actual_full_day', 'Working Days', 'Present', 'Absent'],
      dtype='object')

In [765]:
df_admin_merged.shape

(143, 18)

In [766]:
#holidays_detected = detect_holidays_staffs(
 #   df_admin_in,
  #  year=2025,
   # misc_holidays=HOLIDAY_LIST,   # now list input
    #misc_working_days=misc_working_days,
    #verbose=True
#)

### Step 1.1: Faculty Detailed view (GIMT + GIPS only)

In [767]:
#df_fac_all = pd.concat([splits["GIMT"]["all"], splits["GIPS"]["all"]], ignore_index=True)
df_fac_detail = pd.concat([df_gimt_all, df_gips_all], ignore_index=True)
df_fac_conso = pd.concat([df_gimt_merged, df_gips_merged], ignore_index=True)
df_fac_detail.fillna(0, inplace=True)
df_fac_conso.fillna(0, inplace=True)


In [768]:
df_fac_conso.head()

,Emp Id,Name,Designation,Department,late_flags,early_flags,half_day_flags,AM_abs,PM_abs,days_abs,No_of_AM_abs,No_of_PM_abs,actual_No_of_late,actual_half_day,actual_full_day,Working Days,Present,Absent
0,GCU010013,Ms. Raisa,-,Administration,08_28,"08_27, 09_13",,,,"09_12, 09_18",0,0,1,0,2,20,18.0,2.0
1,GCU010017,Ms. Ananya,-,Administration,08_27,,,,,09_03,0,0,1,0,1,20,19.0,1.0
2,GCU020001,Dr. Lipi,Assistant Professor & HoD (I/c),School of Natural Science,,,,,,"08_26, 09_11",0,0,0,0,2,20,18.0,2.0
3,GCU020002,Dr. Swarnali,Assistant Professor & HoD (I/c),School of Natural Science,,,,,,"08_28, 08_29",0,0,0,0,2,20,18.0,2.0
4,GCU020003,Dr. Kajal,Assistant Professor,School of Natural Science,,,,,,,0,0,0,0,0,20,20.0,0.0


In [769]:
df_admin_detail = df_admin_all.copy()
df_admin_detail.fillna(0, inplace=True)  

df_admin_conso = df_admin_merged.copy()
df_admin_conso.fillna(0, inplace=True)  

In [770]:
df_admin_conso.head()

,Emp Id,Name,Designation,Department,late_flags,early_flags,half_day_flags,AM_abs,PM_abs,days_abs,No_of_AM_abs,No_of_PM_abs,actual_No_of_late,actual_half_day,actual_full_day,Working Days,Present,Absent
0,GCU010004,Kishore Kr.,-,Administration,"08_28, 08_29, 08_30, 09_02, 09_03, 09_05, 09_0...",,"08_26, 08_27, 09_01, 09_04, 09_08","08_26, 08_27, 09_01, 09_04",09_08,09_16,4,1,14,5,1,20,16.5,3.5
1,GCU010005,Dwipen,Administration,Administration,"08_30, 09_02, 09_04, 09_09, 09_12, 09_15, 09_19",,,,,,0,0,7,0,0,20,20.0,0.0
2,GCU010011,Nripen Ch,-,Administration,,,,,,,0,0,0,0,0,20,20.0,0.0
3,GCU010012,Sourabh Mazinder,-,Administration,,,,,,,0,0,0,0,0,20,20.0,0.0
4,GCU010024,Mridul,-,Administration,09_09,,09_15,,09_15,,0,1,1,1,0,20,19.5,0.5


In [771]:
#['Emp Id', 'Name', 'Designation', 'Department', 'late_flags',
 #      'early_flags', 'half_day_flags', 'AM_abs', 'PM_abs', 'days_abs',
  #     'No_of_AM_abs', 'No_of_PM_abs', 'No_of_late', 'HD Absent', 'FD Absent',
   #    'Working Days', 'Present'],

In [772]:
df_fac_conso.columns

Index(['Emp Id', 'Name', 'Designation', 'Department', 'late_flags',
       'early_flags', 'half_day_flags', 'AM_abs', 'PM_abs', 'days_abs',
       'No_of_AM_abs', 'No_of_PM_abs', 'actual_No_of_late', 'actual_half_day',
       'actual_full_day', 'Working Days', 'Present', 'Absent'],
      dtype='object')

In [773]:
df_admin_conso.columns

Index(['Emp Id', 'Name', 'Designation', 'Department', 'late_flags',
       'early_flags', 'half_day_flags', 'AM_abs', 'PM_abs', 'days_abs',
       'No_of_AM_abs', 'No_of_PM_abs', 'actual_No_of_late', 'actual_half_day',
       'actual_full_day', 'Working Days', 'Present', 'Absent'],
      dtype='object')

In [774]:
    col_to_rename = {'AM_abs':'actual_AM_abs','PM_abs':'actual_PM_abs','days_abs':'actual_days_abs','No_of_late':'actual_No_of_late'}

    df_fac_conso_ID.rename(columns=col_to_rename, inplace=True)
    df_admin_conso_ID.rename(columns=col_to_rename, inplace=True)


In [775]:
#df_fac_conso.columns
# current dataframe: df
desired_order = [
    'Emp Id', 'Name','Designation','Department',
    'Working Days', 'Present', 'Absent',
    'actual_half_day','actual_full_day','actual_No_of_late'
]

# Keep only those that exist in df (to avoid KeyError)
desired_order = [col for col in desired_order if col in df_fac_conso.columns]
desired_order = [col for col in desired_order if col in df_admin_conso.columns]

df_fac_conso = df_fac_conso[desired_order]
df_admin_conso = df_admin_conso[desired_order]

In [776]:
df_admin_conso.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,actual_half_day,actual_full_day,actual_No_of_late
0,GCU010004,Kishore Kr.,-,Administration,20,16.5,3.5,5,1,14
1,GCU010005,Dwipen,Administration,Administration,20,20.0,0.0,0,0,7
2,GCU010011,Nripen Ch,-,Administration,20,20.0,0.0,0,0,0
3,GCU010012,Sourabh Mazinder,-,Administration,20,20.0,0.0,0,0,0
4,GCU010024,Mridul,-,Administration,20,19.5,0.5,1,0,1


## Step 2: Merge with ERP employee data

In [777]:
df_emp.head(3)

,Serial No,Employee ID,Name,Department,Group Name,User Type,Designation,Joining Date,Email ID,Mobile No.,Date of Birth,Sub Department,Academy
0,1,APPLICANT_PORTAL_USER,APP_FIRST_NAME APP_MIDDLE_NAME APP_LAST_NAME,-,Admin,-,-,-,-,-,-,-,Girijananda Chowdhury University
1,2,mis,MIS Admin,Administration,Admin,-,-,-,mayank.pandya@serosoft.in,+91-1111111111,18/03/1999,-,Girijananda Chowdhury University
2,3,EMP4,Lokesh Yadav,School of Allied Health Sciences,Admin,-,-,-,lokesh.yadav@serosoft.in,+91-1111111111,09/01/1989,-,Girijananda Chowdhury University


In [778]:
emp_df = df_emp[['Employee ID','Name']]
emp_df = emp_df.rename(columns={'Employee ID':'Emp Id'})

In [779]:
#df_fac_detail.rename(columns={'Emp Id':'Emp ID'}, inplace=True)
#df_admin_detail.rename(columns={'Emp Id':'Emp ID'},inplace=True)

In [780]:
df_fac_detail_ID = pd.merge(df_fac_detail, emp_df, how='left', on='Emp Id')
df_fac_detail_ID = move_columns(df_fac_detail_ID, {'Name':1,'Designation':2,'Department':3})
df_fac_detail_ID = df_fac_detail_ID.drop('Names', axis=1)

df_admin_detail_ID = pd.merge(df_admin_detail, emp_df, how='left', on='Emp Id')
df_admin_detail_ID = move_columns(df_admin_detail_ID, {'Name':1,'Designation':2,'Department':3})
df_admin_detail_ID = df_admin_detail_ID.drop('Names', axis=1)


In [781]:
df_admin_conso.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,actual_half_day,actual_full_day,actual_No_of_late
0,GCU010004,Kishore Kr.,-,Administration,20,16.5,3.5,5,1,14
1,GCU010005,Dwipen,Administration,Administration,20,20.0,0.0,0,0,7
2,GCU010011,Nripen Ch,-,Administration,20,20.0,0.0,0,0,0
3,GCU010012,Sourabh Mazinder,-,Administration,20,20.0,0.0,0,0,0
4,GCU010024,Mridul,-,Administration,20,19.5,0.5,1,0,1


In [782]:
df_fac_conso.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,actual_half_day,actual_full_day,actual_No_of_late
0,GCU010013,Ms. Raisa,-,Administration,20,18.0,2.0,0,2,1
1,GCU010017,Ms. Ananya,-,Administration,20,19.0,1.0,0,1,1
2,GCU020001,Dr. Lipi,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0,2,0
3,GCU020002,Dr. Swarnali,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0,2,0
4,GCU020003,Dr. Kajal,Assistant Professor,School of Natural Science,20,20.0,0.0,0,0,0


In [783]:
# there was error till here

In [784]:
if 'Name' in df_fac_conso.columns:
    df_fac_conso = df_fac_conso.drop(columns='Name', axis=1)
if 'Name' in df_admin_conso.columns:
    df_admin_conso = df_admin_conso.drop(columns='Name', axis=1)


df_fac_conso_ID = pd.merge(df_fac_conso, emp_df, how='left', on='Emp Id')
df_fac_conso_ID = move_columns(df_fac_conso_ID, {'Name':1,'Designation':2,'Department':3})
#df_fac_conso_ID = df_fac_conso_ID.drop('Names', axis=1)

df_admin_conso_ID = pd.merge(df_admin_conso, emp_df, how='left', on='Emp Id')
df_admin_conso_ID = move_columns(df_admin_conso_ID, {'Name':1,'Designation':2,'Department':3})
#df_admin_conso_ID = df_admin_conso_ID.drop('Names', axis=1)


In [785]:
df_admin_conso_ID.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,actual_half_day,actual_full_day,actual_No_of_late
0,GCU010004,Kishore Kumar Choudhury,-,Administration,20,16.5,3.5,5,1,14
1,GCU010005,Dwipen Das,Administration,Administration,20,20.0,0.0,0,0,7
2,GCU010011,Nripen Chandra Barman,-,Administration,20,20.0,0.0,0,0,0
3,GCU010012,Sourobh Mazinder Barua,-,Administration,20,20.0,0.0,0,0,0
4,GCU010024,Mridul Das,-,Administration,20,19.5,0.5,1,0,1


### SAVE 1 & 2 (Detailed and Consolidated data)

In [786]:
# 1
print(f"This is the detailed faculty biometric data {df_fac_detail_ID.shape}")
print(f"This is the detailed admin biometric data {df_admin_detail_ID.shape}")

# 2
print(f"This is the consolidate faculty biometric data {df_fac_conso_ID.shape}")
print(f"This is the consolidate admin biometric data {df_admin_conso_ID.shape}")

This is the detailed faculty biometric data (161, 54)
This is the detailed admin biometric data (143, 54)
This is the consolidate faculty biometric data (161, 10)
This is the consolidate admin biometric data (143, 10)


### 2.1 Handling half days

In [787]:
#df_admin_with_ID.to_excel("./output/staff_bio_data.xlsx", index=False)

In [788]:
# helper for safety
#def safe_len(x):
#    return len(x) if isinstance(x, (list, str)) else 0

In [789]:
df_fac_actual = df_fac_conso_ID.copy()
df_admin_actual = df_admin_conso_ID.copy()

def count_unique_half_days(row):
    am_abs = set(row.get('AM_abs', []) if 'AM_abs' in row else row.get('actual_AM_abs', [])) if isinstance(row.get('AM_abs', []) if 'AM_abs' in row else row.get('actual_AM_abs', []), list) else set()
    pm_abs = set(row.get('PM_abs', []) if 'PM_abs' in row else row.get('actual_PM_abs', [])) if isinstance(row.get('PM_abs', []) if 'PM_abs' in row else row.get('actual_PM_abs', []), list) else set()
    days_abs = set(row.get('days_abs', []) if 'days_abs' in row else row.get('actual_days_abs', [])) if isinstance(row.get('days_abs', []) if 'days_abs' in row else row.get('actual_days_abs', []), list) else set()
    # Half days are those in AM or PM abs but NOT in full day abs
    half_days = (am_abs | pm_abs) - days_abs
    return len(half_days)

# handling half days
df_fac_actual['actual_half_day'] = df_fac_actual.apply(count_unique_half_days, axis=1)
df_admin_actual['actual_half_day'] = df_admin_actual.apply(count_unique_half_days, axis=1)

# handling half days
#df_fac_actual['actual_half_day'] = df_fac_actual.apply(lambda x: len(x['actual_AM_abs'])+len(x['actual_PM_abs']),axis=1)
#df_admin_actual['actual_half_day'] = df_admin_actual.apply(lambda x: len(x['actual_AM_abs'])+len(x['actual_PM_abs']),axis=1)

# handling full days
#df_fac_actual['actual_full_day'] = df_fac_actual.apply(lambda x: len(x['actual_days_abs']),axis=1)
#df_admin_actual['actual_full_day'] = df_admin_actual.apply(lambda x: len(x['actual_days_abs']),axis=1)

# handling half days
#df_fac_actual['actual_half_day'] = df_fac_actual['actual_AM_abs']+ df_fac_actual['actual_PM_abs']
#df_admin_actual['actual_half_day'] = df_admin_actual['actual_AM_abs']+df_admin_actual['actual_PM_abs']

# handling full days
#df_fac_actual['actual_full_day'] = df_fac_actual['actual_days_abs'])
#df_admin_actual['actual_full_day'] = df_admin_actual.apply(lambda x: len(x['actual_days_abs']),axis=1)

In [790]:
df_fac_actual.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,actual_half_day,actual_full_day,actual_No_of_late
0,GCU010013,Raisa Choudhury,-,Administration,20,18.0,2.0,0,2,1
1,GCU010017,Ananya Borah,-,Administration,20,19.0,1.0,0,1,1
2,GCU020001,Lipi Goswami,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0,2,0
3,GCU020002,Swarnali Pathak,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0,2,0
4,GCU020003,Kajal Dutta,Assistant Professor,School of Natural Science,20,20.0,0.0,0,0,0


In [791]:
df_admin_actual.columns   # ['Working Days', 'Present', 'Absent']

Index(['Emp Id', 'Name', 'Designation', 'Department', 'Working Days',
       'Present', 'Absent', 'actual_half_day', 'actual_full_day',
       'actual_No_of_late'],
      dtype='object')

In [792]:
#col_to_select = ['Emp Id', 'Name', 'Designation', 'Department', 'Working Days', 'Present', 'Absent',\
#                               'actual_half_day', 'actual_full_day', 'actual_No_of_late']

#df_fac_actual = df_fac_actual[col_to_select]
#df_admin_actual = df_admin_actual[col_to_select]


In [793]:
df_fac_actual.head(3)

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,actual_half_day,actual_full_day,actual_No_of_late
0,GCU010013,Raisa Choudhury,-,Administration,20,18.0,2.0,0,2,1
1,GCU010017,Ananya Borah,-,Administration,20,19.0,1.0,0,1,1
2,GCU020001,Lipi Goswami,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0,2,0


In [794]:
df_admin_actual.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,actual_half_day,actual_full_day,actual_No_of_late
0,GCU010004,Kishore Kumar Choudhury,-,Administration,20,16.5,3.5,0,1,14
1,GCU010005,Dwipen Das,Administration,Administration,20,20.0,0.0,0,0,7
2,GCU010011,Nripen Chandra Barman,-,Administration,20,20.0,0.0,0,0,0
3,GCU010012,Sourobh Mazinder Barua,-,Administration,20,20.0,0.0,0,0,0
4,GCU010024,Mridul Das,-,Administration,20,19.5,0.5,0,0,1


## Step 3: Exempted leave adjustments

In [795]:
#df_exempted = pd.read_excel('./data/template_exempted.xlsx')
df_exempted = process_exempted_leaves(exempted_file)

In [796]:
df_exempted.head()

,Emp Id,Name,late_count,half_day_count,full_day_count
0,GCU010005,Dwipen Das,0.0,0.0,0.0
1,GCU010012,Sourabh Mazinder Barua,0.0,0.0,0.0
2,GCU010013,Raisa Choudhury,0.0,0.0,0.0
3,GCU010017,Ananya Borah,2.0,0.0,0.0
4,GCU010024,Mridul Das,0.0,0.0,0.0


In [797]:
#df_exempted.to_excel('/output/staff_exempted.xlsx')
#df_exempted.to_excel("./output/staff_exempted.xlsx", index=False)

In [798]:
df_exempted.rename(columns={'late_count':'exempt_late','half_day_count':'exempt_HD','full_day_count':'exempt_FD'}, inplace=True)
if 'Name' in df_exempted.columns:
    df_exempted.drop('Name',axis=1,inplace=True)
#df_fac_bio_leave = pd.merge(df_fac_with_ID, df_exempted, how='left', on=['Emp ID']).fillna(0)

In [799]:
df_exempted.head(3)

,Emp Id,exempt_late,exempt_HD,exempt_FD
0,GCU010005,0.0,0.0,0.0
1,GCU010012,0.0,0.0,0.0
2,GCU010013,0.0,0.0,0.0


In [800]:
df_exempted[df_exempted['Emp Id']=='GCU010017']

,Emp Id,exempt_late,exempt_HD,exempt_FD
3,GCU010017,2.0,0.0,0.0


### 3.1 Merging Actual and Exemped Leaves

In [801]:
df_fac_actual_exempted = pd.merge(df_fac_actual,df_exempted , how='left', on=['Emp Id'])
df_fac_actual_exempted.fillna(0, inplace=True)

df_admin_actual_exempted = pd.merge(df_admin_actual,df_exempted , how='left', on=['Emp Id'])
df_admin_actual_exempted.fillna(0, inplace=True)


In [802]:
df_fac_actual_exempted.head(5)

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,actual_half_day,actual_full_day,actual_No_of_late,exempt_late,exempt_HD,exempt_FD
0,GCU010013,Raisa Choudhury,-,Administration,20,18.0,2.0,0,2,1,0.0,0.0,0.0
1,GCU010017,Ananya Borah,-,Administration,20,19.0,1.0,0,1,1,2.0,0.0,0.0
2,GCU020001,Lipi Goswami,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0,2,0,0.0,0.0,0.0
3,GCU020002,Swarnali Pathak,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0,2,0,0.0,0.0,0.0
4,GCU020003,Kajal Dutta,Assistant Professor,School of Natural Science,20,20.0,0.0,0,0,0,0.0,0.0,0.0


In [803]:
df_admin_actual_exempted.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,actual_half_day,actual_full_day,actual_No_of_late,exempt_late,exempt_HD,exempt_FD
0,GCU010004,Kishore Kumar Choudhury,-,Administration,20,16.5,3.5,0,1,14,0.0,0.0,0.0
1,GCU010005,Dwipen Das,Administration,Administration,20,20.0,0.0,0,0,7,0.0,0.0,0.0
2,GCU010011,Nripen Chandra Barman,-,Administration,20,20.0,0.0,0,0,0,0.0,0.0,0.0
3,GCU010012,Sourobh Mazinder Barua,-,Administration,20,20.0,0.0,0,0,0,0.0,0.0,0.0
4,GCU010024,Mridul Das,-,Administration,20,19.5,0.5,0,0,1,0.0,0.0,0.0


### 3.2 Calculating the balance Actual and Exempted Leaves

In [804]:
#df_fac_attend.head(3)

In [805]:
df_fac_actual_exempted.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,actual_half_day,actual_full_day,actual_No_of_late,exempt_late,exempt_HD,exempt_FD
0,GCU010013,Raisa Choudhury,-,Administration,20,18.0,2.0,0,2,1,0.0,0.0,0.0
1,GCU010017,Ananya Borah,-,Administration,20,19.0,1.0,0,1,1,2.0,0.0,0.0
2,GCU020001,Lipi Goswami,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0,2,0,0.0,0.0,0.0
3,GCU020002,Swarnali Pathak,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0,2,0,0.0,0.0,0.0
4,GCU020003,Kajal Dutta,Assistant Professor,School of Natural Science,20,20.0,0.0,0,0,0,0.0,0.0,0.0


In [806]:
df_fac_actual_exempted['Half Days'] = np.maximum(df_fac_actual_exempted['actual_half_day'] - df_fac_actual_exempted['exempt_HD'],0)
df_fac_actual_exempted['Full Days'] = np.maximum(df_fac_actual_exempted['actual_full_day'] - df_fac_actual_exempted['exempt_FD'],0)
df_fac_actual_exempted['Late'] = np.maximum(df_fac_actual_exempted['actual_No_of_late'] - df_fac_actual_exempted['exempt_late'],0)

df_admin_actual_exempted['Half Days'] = np.maximum(df_admin_actual_exempted['actual_half_day'] - df_admin_actual_exempted['exempt_HD'],0)
df_admin_actual_exempted['Full Days'] = np.maximum(df_admin_actual_exempted['actual_full_day'] - df_admin_actual_exempted['exempt_FD'],0)
df_admin_actual_exempted['Late'] = np.maximum(df_admin_actual_exempted['actual_No_of_late'] - df_admin_actual_exempted['exempt_late'],0)

In [807]:
df_fac_actual_exempted.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,actual_half_day,actual_full_day,actual_No_of_late,exempt_late,exempt_HD,exempt_FD,Half Days,Full Days,Late
0,GCU010013,Raisa Choudhury,-,Administration,20,18.0,2.0,0,2,1,0.0,0.0,0.0,0.0,2.0,1.0
1,GCU010017,Ananya Borah,-,Administration,20,19.0,1.0,0,1,1,2.0,0.0,0.0,0.0,1.0,0.0
2,GCU020001,Lipi Goswami,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0,2,0,0.0,0.0,0.0,0.0,2.0,0.0
3,GCU020002,Swarnali Pathak,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0,2,0,0.0,0.0,0.0,0.0,2.0,0.0
4,GCU020003,Kajal Dutta,Assistant Professor,School of Natural Science,20,20.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [808]:
df_admin_actual_exempted.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,actual_half_day,actual_full_day,actual_No_of_late,exempt_late,exempt_HD,exempt_FD,Half Days,Full Days,Late
0,GCU010004,Kishore Kumar Choudhury,-,Administration,20,16.5,3.5,0,1,14,0.0,0.0,0.0,0.0,1.0,14.0
1,GCU010005,Dwipen Das,Administration,Administration,20,20.0,0.0,0,0,7,0.0,0.0,0.0,0.0,0.0,7.0
2,GCU010011,Nripen Chandra Barman,-,Administration,20,20.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,GCU010012,Sourobh Mazinder Barua,-,Administration,20,20.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,GCU010024,Mridul Das,-,Administration,20,19.5,0.5,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0


### SAVE 3 (Actual and Exempted Adjusted)

In [809]:
# 3
print(f"This is the adjusted faculty actual and exempted data {df_fac_actual_exempted.shape}")
print(f"This is the adjusted admin actual and exempted data {df_admin_actual_exempted.shape}")

This is the adjusted faculty actual and exempted data (161, 16)
This is the adjusted admin actual and exempted data (143, 16)


In [810]:
df_fac_actual_exempted.columns

Index(['Emp Id', 'Name', 'Designation', 'Department', 'Working Days',
       'Present', 'Absent', 'actual_half_day', 'actual_full_day',
       'actual_No_of_late', 'exempt_late', 'exempt_HD', 'exempt_FD',
       'Half Days', 'Full Days', 'Late'],
      dtype='object')

In [811]:
col_to_select = ['Emp Id', 'Name', 'Designation', 'Department', 'Working Days', 'Present', 'Absent','Half Days','Full Days','Late']
df_fac_attend_adjusted = df_fac_actual_exempted[col_to_select]
df_admin_attend_adjusted = df_admin_actual_exempted[col_to_select]

In [812]:
df_fac_attend_adjusted.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Half Days,Full Days,Late
0,GCU010013,Raisa Choudhury,-,Administration,20,18.0,2.0,0.0,2.0,1.0
1,GCU010017,Ananya Borah,-,Administration,20,19.0,1.0,0.0,1.0,0.0
2,GCU020001,Lipi Goswami,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0.0,2.0,0.0
3,GCU020002,Swarnali Pathak,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0.0,2.0,0.0
4,GCU020003,Kajal Dutta,Assistant Professor,School of Natural Science,20,20.0,0.0,0.0,0.0,0.0


In [813]:
df_admin_attend_adjusted.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Half Days,Full Days,Late
0,GCU010004,Kishore Kumar Choudhury,-,Administration,20,16.5,3.5,0.0,1.0,14.0
1,GCU010005,Dwipen Das,Administration,Administration,20,20.0,0.0,0.0,0.0,7.0
2,GCU010011,Nripen Chandra Barman,-,Administration,20,20.0,0.0,0.0,0.0,0.0
3,GCU010012,Sourobh Mazinder Barua,-,Administration,20,20.0,0.0,0.0,0.0,0.0
4,GCU010024,Mridul Das,-,Administration,20,19.5,0.5,0.0,0.0,1.0


## Step 4: ERP Leave integration

In [814]:
df_leave_erp.head(3)

,Serial No.,Employee ID,Name,Location,Leave Type,Request Date,From Date,To Date,Status,Timeline,Total Days
0,1,GCU020107,Salema Khatun,Girijananda Chowdhury University-Assam,Maternity Leave,20-06-2025,02-07-2025,28-12-2025,Approved,View,180
1,2,GCU010033,Parash Kumar Das,Girijananda Chowdhury University-Assam,Earned Leave,20-08-2025,27-08-2025,28-08-2025,Approved,View,2
2,3,GCU030110,Paresh Das,Girijananda Chowdhury University-Assam,Earned Leave,20-08-2025,28-08-2025,30-08-2025,Approved,View,3


In [815]:
df_leave_erp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271 entries, 0 to 270
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Serial No.    268 non-null    object
 1   Employee ID   266 non-null    object
 2   Name          266 non-null    object
 3   Location      266 non-null    object
 4   Leave Type    266 non-null    object
 5   Request Date  266 non-null    object
 6   From Date     266 non-null    object
 7   To Date       266 non-null    object
 8   Status        266 non-null    object
 9   Timeline      266 non-null    object
 10  Total Days    266 non-null    object
dtypes: object(11)
memory usage: 23.4+ KB


In [816]:
df_leave_erp["From Date"] = df_leave_erp["From Date"].apply(preprocess_date)
df_leave_erp["To Date"] = df_leave_erp["To Date"].apply(preprocess_date)

df_leave_erp["From Date"] = pd.to_datetime(df_leave_erp["From Date"], errors='coerce')
df_leave_erp["To Date"] = pd.to_datetime(df_leave_erp["To Date"], errors='coerce')

In [817]:
# corrected leaves
df_leave_erp_summary = calculate_leave_summary_with_wd_leaves(df_leave_erp, calculate_working_days(df_gimt_in)) 
df_leave_erp_summary.fillna(0, inplace=True)


In [818]:
df_leave_erp_summary[df_leave_erp_summary['Emp Id']=='GCU030123']

,Emp Id,Name,Maternity Leave,Earned Leave,Casual Leave,Duty Leave,Sick Leave,Vacation Leave,Total WD leaves
107,GCU030123,Sumit Basfor,0.0,6.0,0.0,0.0,0.0,0.0,5.0


### 4.1 SAVE 4 (leave approve from ERP)

In [819]:
print(f"This is the Leave approved through ERP {df_leave_erp_summary.shape}")

This is the Leave approved through ERP (191, 9)


In [820]:
df_leave_erp_summary.head(3)

,Emp Id,Name,Maternity Leave,Earned Leave,Casual Leave,Duty Leave,Sick Leave,Vacation Leave,Total WD leaves
0,GCU020107,Salema Khatun,180.0,0.0,0.0,0.0,0.0,0.0,20.0
1,GCU010033,Parash Kumar Das,0.0,2.0,0.0,0.0,0.0,0.0,2.0
2,GCU030110,Paresh Das,0.0,3.0,1.0,0.0,0.0,0.0,3.0


In [821]:
df_leave_erp_summary['Approved leaves (ERP)'] = df_leave_erp_summary['Total WD leaves'] + df_leave_erp_summary['Casual Leave']
cols_to_drop = [
    "Casual Leave", "Sick Leave", "Duty Leave", "Vacation Leave",
    "Maternity Leave", "Earned Leave", "Paternity Leave",'Total WD leaves'
]

df_leave_compact = df_leave_erp_summary.drop(columns=[c for c in cols_to_drop if c in df_leave_erp_summary.columns], errors="ignore")

In [822]:
#df_leave_compact = df_leave_compact[['Emp ID','Name','Total WD leaves','Extraordinary Leave']]
# Select only columns that exist
columns_to_select = ['Emp Id', 'Approved leaves (ERP)']
if 'Name' in df_leave_compact.columns:
    columns_to_select.append('Name')
df_leave_compact = df_leave_compact[columns_to_select]


In [823]:
df_leave_compact.head()

,Emp Id,Approved leaves (ERP),Name
0,GCU020107,20.0,Salema Khatun
1,GCU010033,2.0,Parash Kumar Das
2,GCU030110,4.0,Paresh Das
3,GCU030051,3.0,Prakash Talukdar
4,GCU020127,1.0,Rubina Chowdhury


In [824]:
df_leave_compact.drop(columns='Name', axis=1,inplace=True, errors='ignore')

df_fac_report = pd.merge(df_fac_attend_adjusted, df_leave_compact , how='left', on=['Emp Id'])
df_fac_report.fillna(0, inplace=True)

df_admin_report = pd.merge(df_admin_attend_adjusted, df_leave_compact , how='left', on=['Emp Id'])
df_admin_report.fillna(0, inplace=True)


In [825]:
df_fac_report.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Half Days,Full Days,Late,Approved leaves (ERP)
0,GCU010013,Raisa Choudhury,-,Administration,20,18.0,2.0,0.0,2.0,1.0,3.5
1,GCU010017,Ananya Borah,-,Administration,20,19.0,1.0,0.0,1.0,0.0,1.0
2,GCU020001,Lipi Goswami,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0.0,2.0,0.0,2.0
3,GCU020002,Swarnali Pathak,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0.0,2.0,0.0,2.0
4,GCU020003,Kajal Dutta,Assistant Professor,School of Natural Science,20,20.0,0.0,0.0,0.0,0.0,0.0


In [826]:
df_admin_report.head(3)

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Half Days,Full Days,Late,Approved leaves (ERP)
0,GCU010004,Kishore Kumar Choudhury,-,Administration,20,16.5,3.5,0.0,1.0,14.0,0.0
1,GCU010005,Dwipen Das,Administration,Administration,20,20.0,0.0,0.0,0.0,7.0,0.0
2,GCU010011,Nripen Chandra Barman,-,Administration,20,20.0,0.0,0.0,0.0,0.0,0.0


In [827]:
# Calculate Observed Leaves if columns exist
col_to_sum = ['Half Days','Full Days']
existing_cols = [col for col in col_to_sum if col in df_fac_report.columns]
if existing_cols:
    df_fac_report = weighted_sum_and_replace_columns(df_fac_report, existing_cols, 'Observed Leaves', [0.5,1.0])
else:
    df_fac_report['Observed Leaves'] = 0

existing_cols = [col for col in col_to_sum if col in df_admin_report.columns]
if existing_cols:
    df_admin_report = weighted_sum_and_replace_columns(df_admin_report, existing_cols, 'Observed Leaves', [0.5,1.0])
else:
    df_admin_report['Observed Leaves'] = 0



In [828]:
df_fac_report.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Late,Approved leaves (ERP),Observed Leaves
0,GCU010013,Raisa Choudhury,-,Administration,20,18.0,2.0,1.0,3.5,2.0
1,GCU010017,Ananya Borah,-,Administration,20,19.0,1.0,0.0,1.0,1.0
2,GCU020001,Lipi Goswami,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0.0,2.0,2.0
3,GCU020002,Swarnali Pathak,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0.0,2.0,2.0
4,GCU020003,Kajal Dutta,Assistant Professor,School of Natural Science,20,20.0,0.0,0.0,0.0,0.0


In [829]:
#cols_to_delete = ['actual_haf_day','actual_full_day','actual_No_of_late','exempt_late','exempt_HD', 'exempt_FD','actual_half_day']
cols_to_delete = ['Observed Leaves']

df_fac_report = df_fac_report.drop(columns=[c for c in cols_to_delete if c in df_fac_report.columns], errors="ignore")
df_admin_report = df_admin_report.drop(columns=[c for c in cols_to_delete if c in df_admin_report.columns], errors="ignore")

In [830]:
df_admin_report.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Late,Approved leaves (ERP)
0,GCU010004,Kishore Kumar Choudhury,-,Administration,20,16.5,3.5,14.0,0.0
1,GCU010005,Dwipen Das,Administration,Administration,20,20.0,0.0,7.0,0.0
2,GCU010011,Nripen Chandra Barman,-,Administration,20,20.0,0.0,0.0,0.0
3,GCU010012,Sourobh Mazinder Barua,-,Administration,20,20.0,0.0,0.0,0.0
4,GCU010024,Mridul Das,-,Administration,20,19.5,0.5,1.0,0.0


In [831]:
#df_fac_report = df_fac_report.rename(columns={'Approved Leaves':'Biometric leaves'})
#df_admin_report = df_admin_report.rename(columns={'Approved Leaves':'Biometric leaves'})

In [832]:
df_fac_report.sample(10)

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Late,Approved leaves (ERP)
3,GCU020002,Swarnali Pathak,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0.0,2.0
21,GCU020026,Sangeeta Das,Assistant Professor,School Of Engineering and Technology,20,20.0,0.0,0.0,0.5
103,GCU030018,Mahesh Das,Lab Instructor,School Of Engineering and Technology,20,17.0,3.0,1.0,3.0
146,GCU020134,PRAYASH BORAH,Assistant Professor,School of Allied Health Sciences,20,20.0,0.0,0.0,0.0
91,GCU020173,Dipunja Gohain,-,School of Natural Science,20,19.0,1.0,2.0,2.0
12,GCU020014,Debarshi Mallick,Associate Professor & HOD,School Of Engineering and Technology,20,20.0,0.0,0.0,0.0
84,GCU020163,Subungshri Basumatary,Assistant Professor,School Of Engineering and Technology,20,16.5,3.5,4.0,0.0
145,GCU020114,Nayana Adhikari,Assistant Professor,School of Pharmaceutical Science - Guwahati,20,19.0,1.0,0.0,1.0
109,GCU030049,Ranjit Sinha,Lab Instructor,School Of Engineering and Technology,20,20.0,0.0,0.0,0.0
36,GCU020045,NAMRATA KATAKI,Assistant Professor,School Of Engineering and Technology,20,20.0,0.0,1.0,0.5


In [833]:
df_fac_report.columns

Index(['Emp Id', 'Name', 'Designation', 'Department', 'Working Days',
       'Present', 'Absent', 'Late', 'Approved leaves (ERP)'],
      dtype='object')

In [834]:
df_fac_report["Unauthorised leaves"] = (df_fac_report["Absent"] - df_fac_report["Approved leaves (ERP)"]).clip(lower=0)
df_admin_report["Unauthorised leaves"] = (df_admin_report["Absent"] - df_admin_report["Approved leaves (ERP)"]).clip(lower=0)

In [835]:
df_admin_report.sample(10)

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Late,Approved leaves (ERP),Unauthorised leaves
4,GCU010024,Mridul Das,-,Administration,20,19.5,0.5,1.0,0.0,0.5
100,GCU030114,Deepa Das,Non Teaching,Admin Non Teaching,20,18.0,2.0,3.0,1.0,1.0
80,GCU030089,Hanufa Bibi,Cleaner,Admin Non Teaching,20,16.0,4.0,0.0,0.0,4.0
63,GCU030066,Mollika Das,Non Teaching,Admin Non Teaching,20,20.0,0.0,2.0,0.0,0.0
54,GCU030050,Hari Das,Non Teaching,Admin Non Teaching,20,19.0,1.0,0.0,1.5,0.0
93,GCU030107,Diganta Deka,Non Teaching,Admin Non Teaching,20,19.0,1.0,0.0,1.0,0.0
133,GCU030172,0,0,0,20,10.5,9.5,17.0,0.0,9.5
57,GCU030053,Sikha Rani Das,Non Teaching,Admin Non Teaching,20,17.0,3.0,0.0,2.0,1.0
59,GCU030056,Lakhi Nath Barman,Driver,Admin Non Teaching,20,19.5,0.5,0.0,0.0,0.5
30,GCU030016,Bijuli Das,Cleaner,Admin Non Teaching,20,19.0,1.0,0.0,1.0,0.0


### 4.2 SAVE 5 - Adjusting Approved leaves (ERP) and Observed Leaves

In [836]:
print(f"This is Adjusting Approved leaves (ERP) and Observed Leaves {df_fac_report.shape}")
print(f"This is Adjusting Approved leaves (ERP) and Observed Leaves {df_admin_report.shape}")

This is Adjusting Approved leaves (ERP) and Observed Leaves (161, 10)
This is Adjusting Approved leaves (ERP) and Observed Leaves (143, 10)


## Step 5: Save to Excel

In [837]:
df_fac_report_print = df_fac_report.copy()
df_admin_report_print = df_admin_report.copy()

# Drop Observed Leaves and Name duplicates if they exist
df_fac_report_print = df_fac_report_print.drop(columns=['Observed Leaves', 'Name_x', 'Name_y'], errors='ignore')
df_admin_report_print = df_admin_report_print.drop(columns=['Observed Leaves', 'Name_x', 'Name_y'], errors='ignore')

df_fac_report_print = df_fac_report_print.rename(columns={'Approved leaves (ERP)': 'Approved leaves'})
df_admin_report_print = df_admin_report_print.rename(columns={'Approved leaves (ERP)': 'Approved leaves'})

### 5.1 SAVE 6 - Final Report

In [838]:
print(f"This is the final report {df_fac_report_print.shape}")
print(f"This is the final report {df_admin_report_print.shape}")

This is the final report (161, 10)
This is the final report (143, 10)


In [839]:
df_admin_report_print.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Late,Approved leaves,Unauthorised leaves
0,GCU010004,Kishore Kumar Choudhury,-,Administration,20,16.5,3.5,14.0,0.0,3.5
1,GCU010005,Dwipen Das,Administration,Administration,20,20.0,0.0,7.0,0.0,0.0
2,GCU010011,Nripen Chandra Barman,-,Administration,20,20.0,0.0,0.0,0.0,0.0
3,GCU010012,Sourobh Mazinder Barua,-,Administration,20,20.0,0.0,0.0,0.0,0.0
4,GCU010024,Mridul Das,-,Administration,20,19.5,0.5,1.0,0.0,0.5


## Verification

In [840]:
# From ERP
sample = 'GCU020038'
print(f"From Leave from ERP :\n {df_leave_compact[df_leave_compact['Emp Id']==sample]}")
print(f"From Leave from Excempted :\n {df_exempted[df_exempted['Emp Id']==sample]}")

From Leave from ERP :
        Emp Id  Approved leaves (ERP)
25  GCU020038                    3.0
From Leave from Excempted :
        Emp Id  exempt_late  exempt_HD  exempt_FD
47  GCU020038          0.0        0.0        0.0


In [841]:
df_fac_report[df_fac_report['Emp Id']== sample]

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Late,Approved leaves (ERP),Unauthorised leaves
32,GCU020038,Priti Sarmah,Assistant Professor,School Of Humanities & Social Science,20,16.5,3.5,4.0,3.0,0.5


In [842]:
sheets_fac = ['Bio details','Bio Consolidated','Exempted','ERP Leave','ERP-Observed Leaves','Report']
dataframes_fac = [df_fac_detail_ID, df_fac_conso_ID, df_fac_actual_exempted, df_leave_erp_summary,df_fac_report,df_fac_report_print]

with pd.ExcelWriter('./output/report_faculties_26_oct_v2.xlsx', engine='xlsxwriter') as writer:
    for s, frame in zip(sheets_fac, dataframes_fac):
        frame.to_excel(writer, sheet_name=s, index=False)
print('✅ Faculties Excel report saved')

✅ Faculties Excel report saved


In [843]:
sheets_admin = ['Bio details','Bio Consolidated','Exempted','ERP Leave','ERP-Observed Leaves','Report']
dataframes_admin = [df_admin_detail_ID, df_admin_conso_ID, df_admin_actual_exempted,df_leave_erp_summary,df_admin_report,df_admin_report_print]

with pd.ExcelWriter('./output/report_staffs_26_oct_v2.xlsx', engine='xlsxwriter') as writer:
    for s, frame in zip(sheets_admin, dataframes_admin):
        frame.to_excel(writer, sheet_name=s, index=False)
print('✅ Staffs Excel report saved')

✅ Staffs Excel report saved
